# LLaMA From Scratch

**References**
- *Coding LLaMA 2 from scratch in PyTorch - KV Cache, Grouped Query Attention, Rotary PE, RMSNorm: [Youtube Video](https://youtu.be/oM4VmoabDAI?si=JtlNl00nZeIOkWxx), [Code](https://github.com/hkproj/pytorch-llama)*
- *LLaMA explained: KV-Cache, Rotary Positional Embedding, RMS Norm, Grouped Query Attention, SwiGLU: [Youtube Video](https://youtu.be/Mn_9W1nCFLo?si=4xJy4OzpPX5YxGqx)*
- *RoFormer: Enhanced Transformer with Rotary Position Embedding: [Paper](https://arxiv.org/abs/2104.09864)*
- *Root Mean Square Layer Normalization: [Paper](https://arxiv.org/abs/1910.07467)*
- *GQA: Training Generalized Multi-Query Transformer Models from Multi-Head Checkpoints: [Paper](https://arxiv.org/abs/2305.13245)*

## Imports

In [2]:
from dataclasses import dataclass
from typing import Optional, List, Dict, Any
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from pathlib import Path
import json
from sentencepiece import SentencePieceProcessor
from tqdm import tqdm
import time

# Training

### LLaMA Family

**Model Architecture**

<table>
    <tr>
        <td></td>
        <td><strong>Training Data</strong></td>
        <td><strong>Params</strong></td>
        <td><strong>Context length</strong></td>
        <td><strong>GQA</strong></td>
        <td><strong>Token count</strong></td>
    </tr>
    <tr>
        <td rowspan="4">Llama 1</td>
        <td rowspan="4">See Touvron et al. (2023)</td>
        <td>7B</td>
        <td>2k</td>
        <td>✗</td>
        <td>1T+</td>
    </tr>
    <tr>
        <td>13B</td>
        <td>2k</td>
        <td>✗</td>
        <td>1T+</td>
    </tr>
    <tr>
        <td>33B</td>
        <td>2k</td>
        <td>✗</td>
        <td>1.4T+</td>
    </tr>
    <tr>
        <td>65B</td>
        <td>2k</td>
        <td>✗</td>
        <td>1.4T+</td>
    </tr>
    <tr>
        <td rowspan="4">Llama 2</td>
        <td rowspan="4">A new mix of publicly available online data.</td>
        <td>7B</td>
        <td>4k</td>
        <td>✗</td>
        <td rowspan="4">2T+</td>
    </tr>
    <tr>
        <td>13B</td>
        <td>4k</td>
        <td>✗</td>
    </tr>
    <tr>
        <td>34B</td>
        <td>4k</td>
        <td>✔️</td>
    </tr>
    <tr>
        <td>70B</td>
        <td>4k</td>
        <td>✔️</td>
    </tr>
    <tr>
        <td rowspan="2">Llama 3</td>
        <td rowspan="2">A new mix of publicly available online data.</td>
        <td>8B</td>
        <td>8k</td>
        <td>✔️</td>
        <td rowspan="2">15T+</td>
    </tr>
    <tr>
        <td>70B</td>
        <td>8k</td>
        <td>✔️</td>
    </tr>
</table>

### Model Arguments



In [2]:
@dataclass
class ModelArgs:
    dim: int = 4096
    n_layers: int = 32
    # * Unlike the og transformer, we don't need to have the same q, k, v values in LLaMA
    n_heads: int = 32  # number of heads for the queries
    n_kv_heads: Optional[int] = None  # Number of heads for the keys and values
    vocab_size: int = -1  # will be set when we load the tokenizer
    # * since grouped query attention heads are reduced,
    # * the number of params in the FFN is increased to keep the total number of parameters the same
    multiple_of: int = 256
    ffn_dim_multiplier: Optional[float] = None
    norm_eps: float = 1e-5  # epsilon for layer norm

    # needed for KV cache
    max_batch_size: int = 32
    max_seq_len: int = 2048

    device: str = None

## Rotary Positional Embedding

### Precompute Theta Posistional Frequencies

Below are the steps involved in precomputing theta positional frequencies:

![Precompute Theta Posistional Frequencies Steps](images/theta-pos-freq-steps.png)

In [3]:
def precompute_theta_pos_frequencies(
    head_dim: int, seq_len: int, device: str, theta: float = 10000.0
):
    # theta 10000.0 is the default value in the paper
    # As written in the paragraph 3.2.2 of the paper
    # >> In order to generalize our results in 2D to any xi ∈ Rd where **d is even**, [...]
    assert (
        head_dim % 2 == 0
    ), "Dimension must be even since rotary embedding can't be applied to odd."

    # Build the theta parameter
    # According to the formula theta_i = 10000^(-2(i-1)/dim) for i = [1, 2, ..., dim/2]
    theta_numerator = torch.arange(0, head_dim, 2).float()  # (head_dim / 2)
    theta = 1.0 / (theta ** (theta_numerator / head_dim)).to(device)  # (dim / 2)
    # construct the positions (the "m" parameter)
    m = torch.arange(seq_len, device=device)  # (seq_len)
    # Multiply each theta by each position using the outer product.
    # (seq_len), outer_product*(head_dim/2) -> (seq_len,head_dim/2)
    freqs = torch.outer(m, theta).float()
    # we can compute complex numbers in the polar form c = R*exp(m*theta), where R=1 as follow:
    freqs_complex = torch.polar(torch.ones_like(freqs), freqs)
    return freqs_complex

### Rotary Embeddings

The Steps in calculating the Rotary Embedding:
![The Steps in calculating the Rotary Embedding](images/rotary-embedding-steps.png)

Figure 1: Implementation of Rotary Position Embedding(RoPE):
![Implementation of Rotary Position Embedding](images/implementation-of-rope.png)


In [4]:
def apply_rotary_embeddings(x: torch.Tensor, freqs_complex: torch.Tensor, device: str):
    # * OP 1 & 2 >>
    # seperate the last dimension pairs of 2 values, representing the real & imaginary parts of the complex number
    # two consecutive values will become a single complex number
    # (B,seq_len,H,head_dim) -> (B,seq_len,H,head_dim/2)
    x_complex = torch.view_as_complex(x.float().reshape(*x.shape[:-1], -1, 2))
    # reshape the freqs_complex tensor to match the shape of the x_complex tensor.
    # So we need to add the batch dimension and the head dimension.
    # (seq_len,head_dim/2) -> (1,seq_len,1,head_dim/2)
    freqs_complex = freqs_complex.unsqueeze(0).unsqueeze(2)
    # * OP 3 >>
    # Multiply each complex number in the x_complex tensor by the corresponding complex number in the freqs_complex tensor
    # which results in the rotation of the complex number as shown in the Figure 1 of the paper.
    # (B,seq_len,H,head_dim/2)*(1,seq_len,1,head_dim/2) -> (B,seq_len,H,head_dim/2)
    x_rotated = x_complex * freqs_complex
    # * OP 4 >> convert the complex number back to the real number
    # (B,seq_len,H,head_dim/2) -> (B,seq_len,H,head_dim/2,2)
    x_out = torch.view_as_real(x_rotated)
    # * OP 5 >> Flattening to the shape of the original tensor
    # (B,seq_len,H,head_dim/2,2) -> (B,seq_len,H,head_dim)
    x_out = x_out.reshape(*x.shape)
    return x_out.type_as(x).to(device)

## Root Mean Square Normalization

LayerNorm works because of its re-centering and re-scaling invariance property. Re-centering enables the model to be insensitive to shift noises on both inputs and weights, and re-scaling keeps the output representations intact when both inputs and weights are randomly scaled. 
RMS Normalizaiton paper hypothesize that the re-scaling invariance is the reason for success of LayerNorm, rather than re-centering invariance and they propose RMSNorm which only focuses on re-scaling invariance and regularizes the summed inputs simply according to the root mean square (RMS) statistic:

$$
\bar{a}_i= \frac{a_i}{RMS(a)} g_i \\
\text{where} \ RMS(a) = \sqrt{\frac{1}{n} \sum_{i=1}^n a_{i}^2}
$$

Intuitively, RMSNorm simplifies LayerNorm by totally removing the mean statistic at the cost of sacrificing the invariance that mean normalization affords.

In [5]:
class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6) -> None:
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))  # The gamma parameter

    def _norm(self, x: torch.Tensor) -> torch.Tensor:
        # rsqrt: 1 / sqrt(x)
        # (B,seq_len,dim)*(B,seq_len,1) -> (B,seq_len,dim)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # (dim)*(B,seq_len,dim) -> (B,seq_len,dim)
        return self.weight * self._norm(x.float()).type_as(x)

## Feed Forward

### SwiGLU Activation Function

In [6]:
class FeedForward(nn.Module):
    def __init__(self, args: ModelArgs) -> None:
        super().__init__()

        hidden_dim = 4 * args.dim
        hidden_dim = int(2 * hidden_dim / 3)
        if args.ffn_dim_multiplier is not None:
            hidden_dim = int(args.ffn_dim_multiplier * hidden_dim)
        # Round the hidden_dim to the nearest multiple of the multiple_of parameter
        hidden_dim = args.multiple_of * (
            (hidden_dim + args.multiple_of - 1) // args.multiple_of
        )

        self.w1 = nn.Linear(args.dim, hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, args.dim, bias=False)
        self.w3 = nn.Linear(args.dim, hidden_dim, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        swish = F.silu(self.w1(x))  # (B, seq_len, dim) -> (B, seq_len, hidden_dim)
        x_v = self.w3(x)  # (B, seq_len, dim) -> (B, seq_len, hidden_dim)
        # (B, seq_len, hidden_dim) * (B, seq_len, hidden_dim) -> (B, seq_len, hidden_dim)
        x = swish * x_v
        x = self.w2(x)  # (B, seq_len, hidden_dim) -> (B, seq_len, dim)
        return x

## Attention

### Repeat KV Cache

Self-Attention during Next Token Prediction Task at Inference T=1:

![Self-Attention during Next Token Prediction Task at T1](images/Self-Attention-during-NTP-Task-T1.png)

Self-Attention during Next Token Prediction Task at Inference T=4:

![Self-Attention during Next Token Prediction Task at T4](images/Self-Attention-during-NTP-Task-T4.png)

Where KV Cache is useful:

![Where KV Cache is useful](images/where-kv-cache-come-in.png)

Self-Attention with KV-Cache at Inference T=1:

![Self-Attention with KV-Cache at T1](images/Self-Attention-with-KV-Cache-T1.png)

Self-Attention with KV-Cache at Inference T=4:

![Self-Attention with KV-Cache at T4](images/Self-Attention-with-KV-Cache-T4.png)

In [7]:
def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    batch_size, seq_len, n_kv_heads, head_dim = x.shape
    if n_rep == 1:
        return x
    return (
        x[:, :, None, :]  # (B, seq_len, n_kv_heads, 1, head_dim)
        .expand(
            batch_size, seq_len, n_kv_heads, n_rep, head_dim
        )  # (B, seq_len, n_kv_heads, n_rep, head_dim)
        .reshape(
            batch_size, seq_len, n_kv_heads * n_rep, head_dim
        )  # (B, seq_len, n_kv_heads * n_rep, head_dim)
    )

### Different attention algorithms

#### Vanilla batched Multi-Head Attention

- Multihead Attention as presented in the original paper "Attention is all you need".
- By setting 𝑚 = 𝑛 (sequence length of query = seq. length of keys and values)
- The number of arithmetic operations performed is $O(bnd^2)$
- The total memory involved in the operations, given by the sum of all the tensors involved in the calculations (including the derived ones!) is $O(bnd + bhn^2 + d^2)$ 
- The ratio between the total memory and the number of arithmetic operations is $O (\frac{1}{k} + \frac{1}{bn})$
- In this case, the ratio is much smaller than 1, which means that the number of memory access we are performing is much less than the number of arithmetic operations, so the memory access is not the bottleneck here.

In [1]:
def MultiHeadAttentionBatched():
    d, m, n, b, h, k, v = 512, 10, 10, 32, 8, (512 // 8), (512 // 8)

    X = torch.rand(b, n, d)  # Query
    M = torch.rand(b, m, d)  # Key and Value
    mask = torch.rand(b, h, n, m)
    P_q = torch.rand(h, d, k)  # W_q
    P_k = torch.rand(h, d, k)  # W_k
    P_v = torch.rand(h, d, v)  # W_v
    P_o = torch.rand(h, d, v)  # W_o

    Q = torch.einsum("bnd,hdk->bhnk", X, P_q)
    K = torch.einsum("bmd,hdk->bhmk", M, P_k)
    V = torch.einsum("bmd,hdv->bhmv", M, P_v)

    logits = torch.einsum("bhnk,bhmk->bhnm", Q, K)
    weights = torch.softmax(logits + mask, dim=-1)

    O = torch.einsum("bhnm,bhmv->bhnv", weights, V)
    Y = torch.einsum("bhnv,hdv->bnd", O, P_o)
    return Y

##### Einsum

`torch.einsum(equation, *operands) -> Tensor`

equation (str): The equation string specifies the subscripts for each dimension of the input operands in the same order as the dimensions. It uses letters in [a-zA-Z] to represent subscripts. The subscripts for each operand are separated by a comma (','). For example, 'ij,jk->ik' specifies subscripts for two 2D operands. The subscripts that appear exactly once in the equation will be part of the output, sorted in increasing alphabetical order. Optionally, you can define the output subscripts by adding an arrow ('->') at the end of the equation followed by the subscripts for the output.

Note that for example for Q, if you want bhkn instead of bhnk, you can do that since the operation is always the sum, its just the output dimension is different so essentially something like a reshape.

The torch.einsum() function in PyTorch is a powerful tool that allows you to perform various multi-dimensional, linear algebraic operations in a concise way. Here are some common use cases:

1. **Element-wise multiplication**: You can use `einsum` to perform element-wise multiplication of two tensors, similar to the `*` operator.
    ```python
    A = torch.rand(3, 3)
    B = torch.rand(3, 3)
    result = torch.einsum('ij,ij->ij', A, B)
    ```

2. **Matrix multiplication**: einsum can be used to perform matrix multiplication, similar to `torch.matmul()` or the `@` operator.
    ```python
    A = torch.rand(3, 4)
    B = torch.rand(4, 5)
    result = torch.einsum('ij,jk->ik', A, B)
    ```

3. **Batch matrix multiplication**: If you have a batch of matrices and you want to multiply them, you can use `einsum` to do this in a single operation.
    ```python
    A = torch.rand(10, 3, 4)
    B = torch.rand(10, 4, 5)
    result = torch.einsum('bij,bjk->bik', A, B)
    ```

4. **Dot product**: `einsum` can be used to compute the dot product of two vectors.
    ```python
    a = torch.rand(5)
    b = torch.rand(5)
    result = torch.einsum('i,i->', a, b)
    ```

5. **Sum along a dimension**: You can use `einsum` to sum the elements of a tensor along a specific dimension, similar to `torch.sum()`.
    ```python
    A = torch.rand(3, 4, 5)
    sum_along_dim1 = torch.einsum('ijk->ik', A)
    ```

6. **Complex operations**: `einsum` really shines when you want to perform more complex operations that involve multiple steps and intermediate tensors.


#### Batched Multi-Head Attention with KV cache

- Uses the KV cache to reduce the number of operations performed.
- By setting 𝑚 = 𝑛 (sequence length of query = seq. length of keys and values)
- The number of arithmetic operations performed is $O(bnd^2)$
- The total memory involved in the operations, given by the sum of all the tensors involved in the calculations (including the derived ones!) is $O(bn^2d + nd^2)$
- The ratio between the total memory and the number of arithmetic operations is $O (\frac{n}{d} + \frac{1}{b})$
- When 𝑛 ≈ 𝑑 (the sequence length is close to the size of the embedding vector) or 𝑏 ≈ 1 (the batch size is 1), the ratio becomes 1 and the memory access now becomes the bottleneck of the algorithm. For the batch size is not a problem, since it is generally much higher than 1, while for the 𝑛/𝑑 term, we need to reduce the sequence length. But there’s a better way...


In [ ]:
def MultiHeadSelfAttentionIncremental():
    d, b, h, k, v = 512, 32, 8, (512 // 8), (512 // 8)

    m = 5  # Suppose we have already cached "m" tokens
    prev_K = torch.rand(b, h, m, k)
    prev_V = torch.rand(b, h, m, v)

    X = torch.rand(b, d)  # Query
    M = torch.rand(b, d)  # Key and Value
    P_q = torch.rand(h, d, k)  # W_q
    P_k = torch.rand(h, d, k)  # W_k
    P_v = torch.rand(h, d, v)  # W_v
    P_o = torch.rand(h, d, v)  # W_o

    q = torch.einsum("bd,hdk->bhk", X, P_q)
    K = torch.concat([prev_K, torch.einsum("bd,hdk->bhk", M, P_k).unsqueeze(2)], axis=2)
    V = torch.concat([prev_V, torch.einsum("bd,hdv->bhv", M, P_v).unsqueeze(2)], axis=2)

    logits = torch.einsum("bhk,bhmk->bhnm", q, K)
    weights = torch.softmax(logits, dim=-1)

    O = torch.einsum("bhm,bhmv->bhv", weights, V)
    Y = torch.einsum("bhv,hdv->bd", O, P_o)
    return Y, K, V

#### Multi-Query Attention with KV cache

- We remove the ℎ dimension from the 𝐾 and the 𝑉, while keeping it for the 𝑄. This means that all the different query heads will share the same keys and values.
- The number of arithmetic operations performed is $O(bnd^2)$
- The total memory involved in the operations, given by the sum of all the tensors involved in the calculations (including the derived ones!) is $O(bnd + bn^2k + nd^2)$
- The ratio between the total memory and the number of arithmetic operations is $O (\frac{1}{d} + \frac{n}{dh} + \frac{1}{b})$
- Comparing with the previous approach, we have reduced the expensive term 𝑛/𝑑 by a factor of h.
- The performance gains are important, while the model's quality degrades only a little bit.


In [ ]:
def MultiquerySelfAttentionIncremental():
    d, b, h, k, v = 512, 32, 8, (512 // 8), (512 // 8)

    m = 5  # Suppose we have already cached "m" tokens
    prev_K = torch.rand(b, m, k)
    prev_V = torch.rand(b, m, v)

    X = torch.rand(b, d)  # Query
    M = torch.rand(b, d)  # Key and Value
    P_q = torch.rand(h, d, k)  # W_q
    P_k = torch.rand(d, k)  # W_k
    P_v = torch.rand(d, v)  # W_v
    P_o = torch.rand(d, v)  # W_o

    q = torch.einsum("bd,hdk->bhk", X, P_q)
    K = torch.concat([prev_K, torch.einsum("bd,dk->bk", M, P_k).unsqueeze(1)], axis=1)
    V = torch.concat([prev_V, torch.einsum("bd,dv->bv", M, P_v).unsqueeze(1)], axis=1)

    logits = torch.einsum("bhk,bmk->bhm", q, K)
    weights = torch.softmax(logits, dim=-1)

    O = torch.einsum("bhm,bmv->bhv", weights, V)
    Y = torch.einsum("bhv,hdv->bd", O, P_o)
    return Y, K, V

### Grouped Multi-Query Attention

Grouped Multi-Query Attention is a compromise between Multi-Head Attention and Multi-Query Attention:

![Grouped Multi-Query Attention compared](images/Grouped-Multi-Query-Attention-compared.png)


In [8]:
class Attention(nn.Module):
    def __init__(self, args: ModelArgs) -> None:
        super().__init__()

        # indicates the number of heads for the keys and values
        self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
        # indicates the number of heads for the queries
        self.n_heads_q = args.n_heads
        # indicates how many times the keys and values should be repeated
        self.n_rep = self.n_heads_q // self.n_kv_heads
        # indicates the dimension of each head, i.e the part of the embedding that each head will be responsible for
        self.head_dim = args.dim // args.n_heads

        self.wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False)
        self.wk = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False)
        self.wv = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False)
        self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False)

        self.cache_k = torch.zeros(
            (args.max_batch_size, args.max_seq_len, self.n_kv_heads, self.head_dim)
        )
        self.cache_v = torch.zeros(
            (args.max_batch_size, args.max_seq_len, self.n_kv_heads, self.head_dim)
        )

    def forward(self, x: torch.Tensor, start_pos: int, freqs_complex: torch.Tensor):
        batch_size, seq_len, _ = x.shape  # (B, 1, dim)
        xq = self.wq(x)  # (B, 1, dim) -> (B, 1, H_Q * head_dim)
        xk = self.wk(x)  # (B, 1, dim) -> (B, 1, H_KV * head_dim)
        xv = self.wv(x)  # (B, 1, dim) -> (B, 1, H_KV * head_dim)

        # (B, 1, H_Q * Head_Dim) -> (B, 1, H_Q, Head_Dim)
        xq = xq.view(batch_size, seq_len, self.n_heads_q, self.head_dim)
        # Size is the same for xk & xv: (B, 1, H_KV * Head_Dim) -> (B, 1, H_KV, Head_Dim)
        xk = xk.view(batch_size, seq_len, self.n_kv_heads, self.head_dim)
        xv = xv.view(batch_size, seq_len, self.n_kv_heads, self.head_dim)

        # Size doesn't change for xq & zk: (B, 1, H_Q, head_dim) -> (B, 1, H_Q, head_dim)
        xq = apply_rotary_embeddings(xq, freqs_complex, x.device)
        xk = apply_rotary_embeddings(xk, freqs_complex, x.device)

        # replace the entry in the cache for this token
        self.cache_k[:batch_size, start_pos : start_pos + seq_len] = xk
        self.cache_v[:batch_size, start_pos : start_pos + seq_len] = xv

        # retrieve all the cached keys and values so far
        # Size is the same for keys & values: (B, seq_len_kv, H_KV, head_dim)
        keys = self.cache_k[:batch_size, : start_pos + seq_len]
        values = self.cache_v[:batch_size, : start_pos + seq_len]

        # since every group of Q shares the same K & V heads,
        # just repeat the K & V heads for every Q in the same group.
        # Doesn't look like grouped query attention is being done here since only 70B LLaMA has this feature.
        # So this is just multi-head attention.
        # Size is the same for keys & values:
        # (B, seq_len_kv, H_KV, head_dim) -> (B, seq_len_kv, H_Q, head_dim)
        keys = repeat_kv(keys, self.n_rep)
        values = repeat_kv(values, self.n_rep)

        xq = xq.transpose(1, 2)  # (B, 1, H_Q, head_dim) -> (B, H_Q, 1, head_dim)
        # Size is the same for keys & values:
        # (B, seq_len_kv, H_Q, head_dim) -> (B, H_Q, seq_len_kv, head_dim)
        keys = keys.transpose(1, 2)
        values = values.transpose(1, 2)

        # (B, H_Q, 1, head_dim) @ (B, H_Q, head_dim, seq_len_kv) -> (B, H_Q, 1, seq_len_kv)
        scores = torch.matmul(xq, keys.transpose(2, 3)) / math.sqrt(self.head_dim)
        # (B, H_Q, 1, seq_len_kv) -> (B, H_Q, 1, seq_len_kv)
        scores = F.softmax(scores.float(), dim=-1).type_as(xq)

        # (B, H_Q, 1, seq_len) @ (B, H_Q, seq_len_kv, head_dim) -> (B, H_Q, 1, head_dim)
        output = torch.matmul(scores, values)
        # (B, H_Q, 1, head_dim) -> (B, 1, H_Q, head_dim) -> (B, 1, dim)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
        return self.wo(output)  # (B, 1, dim)

## Encoder Block

In [9]:
class EncoderBlock(nn.Module):
    def __init__(self, args: ModelArgs) -> None:
        super().__init__()

        self.n_head = args.n_heads
        self.dim = args.dim
        self.head_dim = self.dim // self.n_head

        self.attention = Attention(args)
        self.feed_forward = FeedForward(args)

        # Normalization before the attention block
        self.attention_norm = RMSNorm(self.dim, eps=args.norm_eps)
        # Normalization before the feed-forward block
        self.ffn_norm = RMSNorm(self.dim, eps=args.norm_eps)

    def forward(
        self, x: torch.Tensor, start_pos: int, freqs_complex: torch.Tensor
    ) -> torch.Tensor:
        # (B,seq_len,dim) + (B,seq_len,dim) -> (B,seq_len,dim)
        h = x + self.attention.forward(self.attention_norm(x), start_pos, freqs_complex)
        # (B,seq_len,dim) + (B,seq_len,dim) -> (B,seq_len,dim)
        out = h + self.feed_forward.forward(self.ffn_norm(h))
        return out

## LLaMA Model

![Transformer vs LLaMA](images/Transformer-vs-LLaMA.png)

In [10]:
class Transformer(nn.Module):
    def __init__(self, args: ModelArgs) -> None:
        super().__init__()
        assert args.vocab_size != -1, "vocab_size must be set"

        self.args = args
        self.vocab_size = args.vocab_size
        self.n_layers = args.n_layers  # represents Nx in the figure above: 32 layers
        self.tok_embeddings = nn.Embedding(self.vocab_size, args.dim)

        self.layers = nn.ModuleList()
        for _ in range(args.n_layers):
            self.layers.append(EncoderBlock(args))

        self.norm = RMSNorm(args.dim, eps=args.norm_eps)
        self.output = nn.Linear(args.dim, self.vocab_size, bias=False)

        self.freqs_complex = precompute_theta_pos_frequencies(
            self.args.dim // self.args.n_heads,
            self.args.max_seq_len * 2,
            device=self.args.device,
        )

    def forward(self, tokens: torch.Tensor, start_pos: int):
        batch_size, seq_len = tokens.shape  # (B, seq_len)
        assert seq_len == 1, "Only one token at a time can be processed."

        h = self.tok_embeddings(tokens)  # (B, seq_len) -> (B, seq_len, dim)
        # retrieve the pairs (m, theta) corresponding to the positions [start_pos, start_pos + seq_len]
        freqs_complex = self.freqs_complex[start_pos : start_pos + seq_len]

        # consequently apply all the encoder layers
        for layer in self.layers:
            h = layer(h, start_pos, freqs_complex)
        h = self.norm(h)
        output = self.output(h).float()
        return output

# Inference

## Inferencing strategies

### Logits

- The output of the last linear layer in the Transformer model is called logits. The logits represent the unscaled “probabilities”, but they’re not really probabilities because they do not sum up to 1.
- The softmax scales all the logits in such a way that they sum up to 1.
- The output of the softmax is thus a probability distribution over all the words in the vocabulary, that is, each words in the vocabulary will have a probability associated to it.
- But how do we choose the next token, given this distribution? There are many strategies...

### Greedy

- At every step, we choose the token with the maximum probability, which is appended to the input to generate the next token and so on...
- If the initial tokens happens to be the wrong ones, it’s very likely that the next ones will be wrong as well.
- It is easy to implement but performs poorly in practice.

### Beam Search with K=2

- Beam search has a parameter K which when set to 2 will choose the top 2 probabilities.

Beam Search with K=2 at T=1

![Beam Search with K=2 at T=1](images/Beam-Search-with-K2-at-T1.png)

Beam Search with K=2 at T=2

![Beam Search with K=2 at T=2](images/Beam-Search-with-K2-at-T2.png)

Beam Search with K=2 at T=3

![Beam Search with K=2 at T=3](images/Beam-Search-with-K2-at-T3.png)

### Beam Search

- At every step, we keep alive the top K paths, all the others are killed.
- Increases inferencing time, since at every step must explore K possible options.
- Generally, performs better than the greedy strategy.

### Temperature

- The idea is to scale the logits before applying the softmax
- A **low temperature** makes the model **more confident** (the gap between low and high probabilities increases).
- A **high temperature** makes the model **less confident** (the gap between low and high probabilities reduces).
    ```python
    logits = torch.Tensor([-2.5, -3, -0.6])
    torch.softmax(logits, dim=0)  # No temperature
    >>> tensor([0.1206, 0.0731, 0.8063])

    torch.softmax(logits / 0.4, dim=0)  # Low temperature = 0.4 -> High confident
    >>> tensor([0.0086, 0.0025, 0.9890])

    torch.softmax(logits / 5, dim=0)  # High temperature = 5 -> Less confident
    >>> tensor([0.2970, 0.2687, 0.4343])
    ```

### Random Sampling

- We sample from the random distribution that is output from the softmax.
    ```python
    logits = torch.Tensor([-2.5, -3, -0.6])
    distribution = torch.softmax(logits, dim=0)
    distribution
    >>> tensor([0.1206, 0.0731, 0.8063])
    ```
- The first token will be chosen with a probability of 12.06%, the second with a probability of 7.31% and the last one with a probability of 80.63%
- The higher the probability, the more likely the probability of it being chosen.
- **Problem**: with very little probability it may happen that we choose tokens that are total nonsense.


### Top K

- In the random sampling strategy, it may happen that we choose words that have very little probability, which usually indicates that the token is unrelated to the previous ones.
- With Top K, we keep only the top k highest probabilities, so that tokens with very low probabilities will never be chosen.
    ```python
    # sort the logits in decreasing order
    # we can sort the logits directly because the softmax is a monotonous function
    logits, _ = torch.sort(torch.Tensor([-2.5, -3, -2.8, -0.5, -0.6]), descending=True)
    k = 2
    top_k_logits = logits[:k]
    distribution = torch.softmax(top_k_logits, dim=0)
    distribution
    >>> tensor([0.5250, 0.4750])
    ```
- **Problem**: given the following distributions, low-probability tokens still make their way into the top k tokens (k = 2)
- Distribution 1: **0.5, 0.4**, 0.05, 0.025, 0.025
- Distribution 2: **0.9, 0.05**, 0.025, 0.020, 0.005

### Top P

- With Top P, we keep only the tokens with highest probability, such that their cumulative probability is greater than or equal to the parameter p. This way, we get more tokens for distributions that are more “flat” and less tokens for distributions with a very prominent mode.
    ```python
    # sort the logits in descending order
    p = 0.5
    logits, _ = torch.sort(torch.Tensor([-2.5, -3, -2.8, -0.5, -0.6]), descending=True)
    probs = torch.softmax(logits, dim=0)
    print(f"Probabilities: {probs}")

    probs_cumulative = torch.cumsum(probs, dim=0)
    print(f"Cumulative Probabilities: {probs_cumulative}")

    mask = probs_cumulative - probs > p  # Mask for non-top-p positions
    print(f"Cumulative Probabilities (shifted): {probs_cumulative - probs}")
    print(f"mask: {mask}")

    probs[mask] = 0.0  # zero out all non-top-k tokens
    probs.div_(probs.sum(dim=-1, keepdim=True))  # redistribute probabilities among surviving tokens
    print(f"Top P: {probs}")

    >>> Probabilities: tensor([0.4499, 0.4071, 0.0609, 0.0451, 0.0369])
    >>> Cumulative Probabilities: tensor([0.4499, 0.8571, 0.9180, 0.9631, 1.0000])
    >>> Cumulative Probabilities (shifted): tensor([0.0000, 0.4499, 0.8571, 0.9180, 0.9631])
    >>> mask: tensor([False, False,  True,  True,  True])
    >>> Top P: tensor([0.5250, 0.4750, 0.0000, 0.0000, 0.0000])
    ```

## LLaMA Model

In [11]:
class LLaMA:
    def __init__(
        self,
        model: Transformer,
        tokenizer: SentencePieceProcessor,
        model_args: ModelArgs,
    ) -> None:
        self.model = model
        self.tokenizer = tokenizer
        self.args = model_args

    @staticmethod
    def build(
        checkpoints_dir: str,
        tokenizer_path: str,
        load_model: bool,
        max_seq_len: int,
        max_batch_size: int,
        device: str,
    ):
        prev_time = time.time()
        if load_model:
            checkpoints = sorted(Path(checkpoints_dir).glob("*.pth"))
            assert len(checkpoints) > 0, f"No checkpoints found in {checkpoints_dir}"
            ckpt_path = checkpoints[0]
            print(f"Loading model from checkpoint: {ckpt_path}")
            checkpoint = torch.load(ckpt_path, map_location="cpu")
            print(f"Loaded checkpoint in {time.time() - prev_time:.2f} seconds")
            prev_time = time.time()

        with open(Path(checkpoints_dir) / "params.json", "r") as f:
            params = json.loads(f.read())

        model_args: ModelArgs = ModelArgs(
            max_seq_len=max_seq_len,
            max_batch_size=max_batch_size,
            device=device,
            **params,
        )

        tokenizer = SentencePieceProcessor()
        tokenizer.load(tokenizer_path)
        model_args.vocab_size = tokenizer.vocab_size()

        if device == "cuda":
            torch.set_default_tensor_type(torch.cuda.HalfTensor)
        else:
            torch.set_default_tensor_type(torch.BFloat16Tensor)

        model = Transformer(model_args).to(device)

        if load_model:
            # The only unmatched key in the checkpoint is rotary positional encoding freqs
            # because we will create it during inference, so we can remove it
            del checkpoint["rope.freqs"]
            model.load_state_dict(checkpoint, strict=True)
            print(f"Loaded state dict in {time.time() - prev_time:.2f} seconds")

        return LLaMA(model, tokenizer, model_args)

    def text_completion(
        self,
        prompts: List[str],
        device: str,
        temperature: float = 0.6,
        top_p: float = 0.9,
        max_gen_len: Optional[int] = None,
    ):
        if max_gen_len is None:
            max_gen_len = self.args.max_seq_len - 1

        # convert each prompt into tokens
        prompt_tokens = [
            self.tokenizer.encode(prompt, out_type=int, add_bos=True, add_eos=False)
            for prompt in prompts
        ]

        # Make sure the batch size is not too large
        batch_size = len(prompt_tokens)
        assert (
            batch_size <= self.args.max_batch_size
        ), f"Batch size {batch_size} must be less than or equal to the Max batch size: {self.args.max_batch_size}"

        max_prompt_len = max(len(prompt) for prompt in prompt_tokens)
        # Make sure the prompt length is not larger than the max sequence length
        assert (
            max_prompt_len <= self.args.max_seq_len
        ), f"Prompt length {max_prompt_len} must be less than or equal to the Max sequence length: {self.args.max_seq_len}"

        total_len = min(self.args.max_seq_len, max_gen_len + max_prompt_len)

        # create the list that will contain the generated tokens, along with the initial prompt tokens
        pad_id = self.tokenizer.pad_id()
        tokens = torch.full(
            (batch_size, total_len), pad_id, dtype=torch.long, device=device
        )
        for k, t in enumerate(prompt_tokens):
            # populate the initial tokens with the prompt tokens
            tokens[k, : len(t)] = torch.tensor(t, dtype=torch.long, device=device)

        eos_reached = torch.tensor([False] * batch_size, device=device)
        # True if the token is a prompt token, False otherwise
        prompt_tokens_mask = tokens != pad_id

        cur_iterator = tqdm(range(1, total_len), desc="Generating tokens...")
        for cur_pos in cur_iterator:
            with torch.no_grad():
                logits = self.model.forward(tokens[:, cur_pos - 1 : cur_pos], cur_pos)
            if temperature > 0:
                # The temperature is applied before the softmax
                probs = torch.softmax(logits[:, -1] / temperature, dim=-1)
                next_token = self._sample_top_p(probs, top_p)
            else:
                # greedily select the token with the max probability
                next_token = torch.argmax(logits[:, -1], dim=-1)

            next_token = next_token.reshape(-1)
            # only replace token if it is a padding token
            next_token = torch.where(
                prompt_tokens_mask[:, cur_pos], tokens[:, cur_pos], next_token
            )
            tokens[:, cur_pos] = next_token

            # EOS is reached only if we found an EOS token for a padding position
            eos_reached != (
                ~prompt_tokens_mask[:, cur_pos] & (next_token == self.tokenizer.eos_id)
            )
            if all(eos_reached):
                break

        out_tokens = []
        out_text = []
        for prompt_index, current_prompt_tokens in enumerate(tokens.tolist()):
            # cut to the EOS token, if present
            if self.tokenizer.eos_id in current_prompt_tokens:
                eos_idx = current_prompt_tokens.index(self.tokenizer.eos_id)
                current_prompt_tokens = current_prompt_tokens[:eos_idx]
            out_tokens.append(current_prompt_tokens)
            out_text.append(self.tokenizer.decode(current_prompt_tokens))

        return (out_tokens, out_text)

    def _sample_top_p(self, probs: torch.Tensor, p: float):
        probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
        probs_sum = torch.cumsum(probs_sort, dim=-1)  # (B, vocab_size)
        # (substracting "probs_sort" shifts the cumulative sum by 1 position to the right before masking)
        mask = probs_sum - probs_sort > p  # (B, vocab_size)
        # zero out all the probabilities of tokens that are not selected by the top_p
        probs_sort[mask] = 0.0
        # redistribute the probabilities so that they sum up to 1.
        probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
        # sample a token (its index) from the top p distribution
        next_token = torch.multinomial(probs_sort, num_samples=1)
        # get the token position in the vocabulary corresponding to the sampled index
        next_token = torch.gather(probs_idx, -1, next_token)
        return next_token

## Inference Run

In [12]:
# NOTE: If you want to rerun this with limited GPU: I am using 16175MiB, RTX 3080 Ti Laptop GPU
# You will need to restart the kernal cause jupyter notebook doesn't release the GPU memory
# I tried: `torch.cuda.empty_cache()` to release all the GPU memory that can be freed
# and `%reset -f` to clear jupyter notebook variables, but neither worked

torch.manual_seed(42)

model_version = "llama-2-7b"
allow_cuda = True
device = "cuda" if torch.cuda.is_available() and allow_cuda else "cpu"

prompts = [
    "Simply put, the theory of relativity states that ",
    "If Google was an Italian company founded in Milan, it would",
    # Few shot promt
    """Translate English to French:
    sea otter => loutre de mer
    peppermint => menthe poivrée
    plush girafe => girafe peluche
    cheese =>""",
    # Zero shot prompt
    """Tell me if the following person is actually a Jedi knight disguised as human:
    Name: Mukesh Mithrakumar
    Decision: 
    """,
]

model = LLaMA.build(
    checkpoints_dir=f"models/{model_version}",
    tokenizer_path=f"models/{model_version}/tokenizer.model",
    load_model=True,
    max_seq_len=1024,
    max_batch_size=len(prompts),
    device=device,
)

out_tokens, out_texts = model.text_completion(prompts, device, max_gen_len=64)
assert len(out_texts) == len(prompts)
for i in range(len(out_texts)):
    print(f"{out_texts[i]}\n{'-'*50}")

Loading model from checkpoint: models\llama-2-7b\consolidated.00.pth
Loaded checkpoint in 5.99 seconds


c:\Users\mukes\anaconda3\envs\torchdl\lib\site-packages\torch\__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:433.)
  _C._set_default_tensor_type(t)


Loaded state dict in 9.10 seconds


Generating tokens...: 100%|██████████| 112/112 [00:08<00:00, 12.54it/s]

Simply put, the theory of relativity states that 1) the speed of light is constant in all inertial reference frames, and 2) the laws of physics are the same for all inertial reference frames.Ћ The theory of relativity has two versions: the special theory of relativity and the general theory of relativity. The special theory of relativity applies to all physical phenomena in inertial reference frames. The general theory of relativity applies to the phenomena in non-inertial reference frames. The theory
--------------------------------------------------
If Google was an Italian company founded in Milan, it would be the largest company in the world, with a market capitalization of $ 713 billion. The company was founded in 1998 by Larry Page and Sergey Brin, two PhD students at Stanford University.
The company's name is a reference to the number of letters in the word "googol", which is a mathematical term for the number 1 followed by 100 zeros.
The company has its headquarters in Mountain